In [10]:
import pandas as pd
import nltk, re, pprint
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

In [11]:
tweet = "I subscribe to Luca Mezzalira take on state management. And just for you to know.. white beards are class!! Now let me observe your talk."

In [12]:
def preprocess(tweet):
    tokenized_words = nltk.word_tokenize(tweet)
    pos_tagged = nltk.pos_tag(tokenized_words)
    return pos_tagged

In [13]:
preprocessed_tweet = preprocess(tweet)

In [14]:
"""
Chunking:

"""

# This grammar is described in the paper by S. N. Kim,
# T. Baldwin, and M.-Y. Kan.
# Evaluating n-gram based evaluation metrics for automatic
# keyphrase extraction.
# Technical report, University of Melbourne, Melbourne 2010.
grammar = r"""
    NBAR:
        # Nouns and Adjectives, terminated with Nouns
        {<NN.*|JJ>*<NN.*>}

    NP:
        # Noun Phrase Chunking
        {<NBAR>}
        # Above, connected with in/of/etc...
        {<NBAR><IN><NBAR>}
"""
chunker = nltk.RegexpParser(grammar)
tree = chunker.parse(preprocessed_tweet)
#tree.draw()

In [15]:
# Helper function to traverse the graph and extract the NP
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        yield subtree.leaves()

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    #bool(2 <= len(word) <= 40 and word.lower() not in stopwords)
    return word not in stopwords

In [16]:
# Walking the tree and Normalisation
"""

Normalisation may consist of lower-casing words, removing stop-words which appear in many documents 
(i.e. if, the, a…), stemming (i.e. cars → car), and lemmatizing (i.e. drove, drives, rode → drive). 

"""

def get_terms(tree):
    for leaf in leaves(tree):
        term = [ word.lower() for word, tag in leaf if acceptable_word(word) ]
        yield term
        

lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

In [17]:
terms = get_terms(tree)

In [18]:
extracted_keywords = []
for term in terms:
    for word in term:
        extracted_keywords.append(word)
print(extracted_keywords)

['luca', 'mezzalira', 'state', 'management', 'white', 'beards', 'class', 'talk']
